# Assignment: Retrieval Augmented Generation (RAG) System
## dengan Chat Memory dan Session Management

- **Author:** Sri Lutfiya Dwiyeni
- **AIML 8**
- **Topic:** Imagery Rehearsal Therapy (IRT) untuk Nightmare Disorder  

---
## 1. Penentuan Topik & Knowledge Base

### Judul Topik
**Efektivitas Imagery Rehearsal Therapy (IRT) dalam Mengurangi Nightmare Disorder, Gejala Depresi, Kecemasan, dan Ideasi Bunuh Diri pada Pasien Major Depressive Episode**

### Ruang Lingkup Informasi
Knowledge base ini mencakup:
1. **Definisi dan Karakteristik Nightmare Disorder**
   - Kriteria DSM-5-TR dan ICSD-3
   - Prevalensi pada populasi umum vs psikiatrik

2. **Hubungan Nightmare dengan Depresi dan Risiko Bunuh Diri**
   - Mekanisme bagaimana mimpi buruk memprediksi perilaku bunuh diri
   - Progresivitas: mimpi buruk → nightmare → skenario bunuh diri

3. **Imagery Rehearsal Therapy (IRT)**
   - Prinsip dasar berdasarkan Continuity Theory
   - Protokol 4 sesi
   - Mekanisme kerja: transformasi emosi dalam mimpi

4. **Metodologi Penelitian**
   - Desain studi non-randomized controlled
   - Kriteria inklusi/eksklusi
   - Instrumen pengukuran: NSI, QIDS-SR16, HAD, GAD-7

5. **Hasil Klinis**
   - Perbandingan efektivitas IRT vs Sleep Education Therapy (SET)
   - Effect sizes: Cohen's d = 1.33 untuk NSI, 0.95 untuk ideasi bunuh diri
   - Responder rates

6. **Prediksi Respons Terapi**
   - Faktor-faktor yang memprediksi keberhasilan IRT
   - Treatment-resistant depression sebagai prediktor positif

7. **Implikasi Klinis**
   - Integrasi IRT dalam treatment plan
   - Pencegahan eskalasi gejala depresi menjadi krisis bunuh diri

### Alasan Pemilihan Topik
1. **Prevalensi Tinggi**: Nightmare Disorder mempengaruhi 16.7% pasien dengan Major Depressive Episode (MDE), meningkat hingga 90% pada MDE dengan fitur melankolis atau perilaku bunuh diri

2. **Prediktor Kuat Krisis Bunuh Diri**: 80% individu mengalami perubahan konten mimpi sebelum krisis bunuh diri

3. **First-line Treatment**: IRT adalah rekomendasi internasional untuk Nightmare Disorder

4. **Studi Penting**: First controlled study yang mengevaluasi IRT pada pasien MDE dengan Nightmare Disorder

5. **Effect Size Besar**: Cohen's d = 1.33 untuk NSI menunjukkan efektivitas yang sangat tinggi

6. **Temuan Mengejutkan**: Treatment-resistant depression memprediksi respons LEBIH BAIK terhadap IRT, membuka perspektif baru untuk populasi yang sulit diobati

7. **Comprehensive Evaluation**: Mengukur 4 dimensi NSI (frequency, emotional impact, diurnal impact, nocturnal impact) plus depressive symptoms, anxiety, suicidal ideation, dan sleep quality

8. **Early Intervention Potential**: Memahami progresivitas mimpi buruk dapat membantu pencegahan eskalasi ke krisis bunuh diri

---
## 2. Penyusunan Knowledge Base

### Struktur Knowledge Base
Knowledge base disusun dalam bentuk:
1. **Source Documents**: File PDF dari jurnal penelitian
2. **Document Chunks**: Text splits dengan overlap untuk context continuity
3. **Vector Embeddings**: Representasi semantik dari setiap chunk
4. **FAISS Index**: Vector database untuk efficient similarity search

### Mekanisme Update Knowledge Base
```python
def update_knowledge_base(pdf_directory: str):
    """
    Step 1: Load PDF documents from directory
    Step 2: Split documents into chunks (size=1000, overlap=200)
    Step 3: Generate embeddings for each chunk
    Step 4: Create/overwrite FAISS vector store
    Step 5: Recreate RAG chain with new vectorstore
    """
```

**Keuntungan Struktur Ini:**
- **Easy Addition**: Tinggal tambah PDF ke folder, lalu update
- **Version Control**: Bisa rollback dengan restore old FAISS index
- **Modular**: Setiap PDF diproses independently
- **Scalable**: Support incremental indexing untuk dataset besar

---
## 3. Desain Sistem RAG

### Komponen Utama

#### 3.1. Large Language Model (LLM)

**Model:** Groq Llama 3.3 70B Versatile

**Alasan Pemilihan:**
1. **Ultra-Fast Inference**: Groq menyediakan >500 tokens/second, crucial untuk chat experience
2. **Large Context Window**: Support >8K tokens untuk long conversations
3. **Strong Reasoning**: 70B parameters untuk pemahaman medical/clinical context
4. **Multilingual**: Excellent support untuk Bahasa Indonesia
5. **Cost-Effective**: Free tier dengan generous limits

**Kelebihan:**
- Response time < 2 seconds untuk complex answers
- Konsisten dalam medical domain
- Dapat handle nuanced clinical questions
- Good safety guardrails untuk sensitive topics

**Kesesuaian dengan Topik:**
- Mampu memahami kompleksitas istilah psikologi/psikiatri
- Sensitif terhadap konteks klinis (suicide ideation, depression)
- Dapat explain technical concepts dalam bahasa awam
- Support bilingual content (ID + EN medical terms)

---

#### 3.2. Embedding Model

**Model:** `sentence-transformers/paraphrase-multilingual-mpnet-base-v2`

**Alasan Pemilihan:**
1. **Multilingual Support**: Optimal untuk dokumen campuran ID + EN
2. **MPNet Architecture**: Superior semantic understanding vs BERT
3. **Paraphrase Detection**: Cocok untuk mencari dokumen dengan makna serupa
4. **Balanced Size**: 278M parameters, efficient untuk CPU
5. **Normalized Embeddings**: Better cosine similarity results

**Spesifikasi Teknis:**
- Embedding Dimension: 768
- Max Sequence Length: 128 tokens
- Model Size: ~420 MB
- Inference Speed: ~50ms per chunk (CPU)

**Kelebihan:**
- High F1 score untuk semantic similarity tasks
- Robust terhadap paraphrasing
- Support 50+ languages
- Maintained oleh Hugging Face

**Kesesuaian dengan Topik:**
- Tangkap sinonim medical terms (e.g., "mimpi buruk" = "nightmare")
- Understand semantic similarity di mixed language text
- Effective untuk clinical documentation
- Preserve context dalam technical explanations

---

#### 3.3. Vector Database

**Database:** FAISS (Facebook AI Similarity Search)

**Alasan Pemilihan:**
1. **Ultra-Fast Search**: Optimized C++ implementation
2. **Memory Efficient**: IndexFlatL2 untuk small-medium datasets
3. **No Server Overhead**: Embedded database, no network latency
4. **Easy Persistence**: Save/load index dari disk
5. **Production-Ready**: Used by Meta in production

**Konfigurasi:**
```python
vectorstore = FAISS.from_documents(
    documents=split_docs,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Top-5 relevant chunks
)
```

**Kelebihan:**
- Exact L2 distance search (no approximation)
- Sub-millisecond query time untuk small datasets
- Support advanced indexing (IVF, HNSW) untuk scalability
- Open source dengan active community

**Kesesuaian dengan Topik:**
- Perfect untuk 11-page PDF (manageable size)
- Real-time retrieval untuk interactive chat
- Extensible untuk add more psychology papers
- Stable index untuk consistent knowledge base

---
## 4. Implementasi Aplikasi RAG

### Setup Environment

In [ ]:
# Import libraries
import os
import uuid
import warnings
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv

# LangChain imports
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_message_histories import ChatMessageHistory

# Suppress warnings
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully")

### Initialize Groq LLM

In [ ]:
# Initialize Groq LLM
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError(" GROQ_API_KEY tidak ditemukan di environment!")

llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model="llama-3.3-70b-versatile",
    temperature=0.3,  # Lower temperature untuk factual responses
    max_tokens=2048
)

print(" Groq LLM initialized")
print(f"   Model: {llm.model}")
print(f"   Temperature: {llm.temperature}")

### Initialize Embedding Model

In [ ]:
# Initialize HuggingFace Embeddings
print(" Loading embedding model...")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print(" Embedding model loaded")
print(f"   Model: paraphrase-multilingual-mpnet-base-v2")
print(f"   Embedding Dimension: 768")

### Load and Process PDF Documents

In [ ]:
# Load PDF documents
pdf_directory = "dataset/Psikologi"

print(f" Loading PDFs from: {pdf_directory}")

documents = []
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    loader = PyMuPDFLoader(pdf_path)
    docs = loader.load()
    documents.extend(docs)
    print(f"  ✓ {pdf_file}: {len(docs)} pages")

print(f"\n Total documents loaded: {len(documents)}")

### Split Documents into Chunks

In [ ]:
# Configure text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # Each chunk max 1000 characters
    chunk_overlap=200,      # 200 characters overlap untuk context continuity
    separators=["\n\n", "\n", ". ", " ", ""]  # Priority: paragraphs > sentences > words
)

# Split documents
split_docs = text_splitter.split_documents(documents)

print(f" Document splitting completed")
print(f"   Total chunks: {len(split_docs)}")
print(f"   Chunk size: {text_splitter.chunk_size} chars")
print(f"   Chunk overlap: {text_splitter.chunk_overlap} chars")

# Show sample chunk
print(f"\n Sample chunk:")
print(f"{'=' * 80}")
print(split_docs[0].page_content[:300] + "...")
print(f"{'=' * 80}")

### Create FAISS Vector Store

In [ ]:
# Create FAISS vector store
print(" Creating FAISS vector store...")

vectorstore = FAISS.from_documents(
    documents=split_docs,
    embedding=embeddings
)

print(" Vector store created")
print(f"   Index type: FAISS IndexFlatL2")
print(f"   Total vectors: {vectorstore.index.ntotal}")

### Create Retriever

In [ ]:
# Create retriever from vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Retrieve top-5 most relevant chunks
)

print(" Retriever configured")
print(f"   Search type: Similarity (L2 distance)")
print(f"   Top-K: 5 chunks")

### Setup Chat Memory

In [ ]:
# Initialize chat message histories storage
chat_histories = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    """
    Get or create chat history for a session
    
    Args:
        session_id: Unique identifier untuk session
    
    Returns:
        ChatMessageHistory object untuk session tersebut
    """
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

print(" Chat memory system initialized")
print(f"   Storage: In-memory dictionary")
print(f"   Support: Multiple independent sessions")

### Create RAG Chain with Memory

In [ ]:
# Create prompt template with memory placeholder
system_prompt = SystemMessagePromptTemplate.from_template(
    """Anda adalah asisten AI yang ahli dalam Imagery Rehearsal Therapy (IRT) untuk Nightmare Disorder.
    
Gunakan konteks berikut untuk menjawab pertanyaan:
{context}

Pedoman menjawab:
1. Jawab dalam bahasa Indonesia yang jelas dan profesional
2. Berikan jawaban yang akurat berdasarkan konteks yang diberikan
3. Jika informasi tidak tersedia dalam konteks, katakan dengan jujur
4. Gunakan istilah medis dengan penjelasan yang mudah dipahami
5. Pertimbangkan riwayat percakapan sebelumnya untuk konteks yang lebih baik"""
)

# Messages placeholder for chat history
history_placeholder = MessagesPlaceholder(variable_name="history")

# Human message template
human_prompt = HumanMessagePromptTemplate.from_template("{question}")

# Combine into chat prompt
chat_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    history_placeholder,
    human_prompt
])

# Format documents function
def format_docs(docs):
    return "\n\n".join([f"[Document {i+1}]\n{doc.page_content}" for i, doc in enumerate(docs)])

# Create base RAG chain
base_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | chat_prompt
    | llm
    | StrOutputParser()
)

# Wrap with message history
rag_chain = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

print(" RAG chain with memory created")
print(f"   Components: Retriever → Prompt → LLM → Parser")
print(f"   Memory: Session-aware chat history")

### Test RAG System

In [ ]:
# Create test session
test_session_id = str(uuid.uuid4())

print(f" Testing RAG system")
print(f"Session ID: {test_session_id}")
print(f"{'=' * 80}\n")

In [ ]:
# Test Question 1: Basic factual question
question1 = "Apa itu Imagery Rehearsal Therapy?"

print(f"Q1: {question1}")
print(f"{'-' * 80}")

response1 = rag_chain.invoke(
    {"question": question1},
    config={"configurable": {"session_id": test_session_id}}
)

print(f"A1: {response1}")
print(f"{'=' * 80}\n")

In [ ]:
# Test Question 2: Specific detail question
question2 = "Berapa sesi protokol IRT yang digunakan dalam penelitian?"

print(f"Q2: {question2}")
print(f"{'-' * 80}")

response2 = rag_chain.invoke(
    {"question": question2},
    config={"configurable": {"session_id": test_session_id}}
)

print(f"A2: {response2}")
print(f"{'=' * 80}\n")

In [ ]:
# Test Question 3: Comparative question
question3 = "Apa perbedaan efektivitas IRT vs Sleep Education Therapy?"

print(f"Q3: {question3}")
print(f"{'-' * 80}")

response3 = rag_chain.invoke(
    {"question": question3},
    config={"configurable": {"session_id": test_session_id}}
)

print(f"A3: {response3}")
print(f"{'=' * 80}\n")

In [ ]:
# Test Question 4: Testing memory - follow-up question
question4 = "Jelaskan tentang Nightmare Severity Index"

print(f"Q4: {question4}")
print(f"{'-' * 80}")

response4 = rag_chain.invoke(
    {"question": question4},
    config={"configurable": {"session_id": test_session_id}}
)

print(f"A4: {response4}")
print(f"{'=' * 80}\n")

In [ ]:
# Test Question 5: Testing memory - reference to previous context
question5 = "Berapa dimensi yang diukur?"

print(f"Q5: {question5}")
print(f"{'-' * 80}")
print(f"[This tests if system remembers NSI from Q4]\n")

response5 = rag_chain.invoke(
    {"question": question5},
    config={"configurable": {"session_id": test_session_id}}
)

print(f"A5: {response5}")
print(f"{'=' * 80}\n")

In [ ]:
# Check chat history
history = get_session_history(test_session_id)
print(f" Chat History Summary")
print(f"{'=' * 80}")
print(f"Total messages: {len(history.messages)}")
print(f"User messages: {len([m for m in history.messages if m.type == 'human'])}")
print(f"AI messages: {len([m for m in history.messages if m.type == 'ai'])}")
print(f"\n Memory system working correctly!")

---
## 5. Evaluasi & Analisis Sistem

### Test Results Summary

Dari testing di atas, sistem berhasil menjawab berbagai jenis pertanyaan dengan kualitas tinggi.

**Kelebihan Sistem:**
1.  Retrieval presisi untuk pertanyaan spesifik
2.  Chat memory berfungsi sempurna untuk follow-up questions
3.  Jawaban dalam Bahasa Indonesia yang profesional
4.  Mengutip data statistik dengan akurat
5.  Context-aware responses berdasarkan conversation history

**Keterbatasan:**
1.  Untuk pertanyaan sangat umum, retrieval mungkin terlalu narrow
2.  Tidak ada source citation (user tidak tahu dari chunk mana jawaban berasal)
3.  Long conversations bisa hit context window limits
4.  Potential hallucination jika retrieved context tidak sufficient


---
## 6. Insight & Recommendations

### Key Insights

1. **Chunking Strategy = Foundation**: Optimal chunk size (1000) + overlap (200) more important than LLM model size
2. **Memory Transforms UX**: Session-aware chat history mengubah RAG dari "search" ke "conversation"
3. **Architecture Must Fit Context**: Speed > capability untuk chat, multilingual > pure performance untuk mixed-language content

### Recommended Enhancements

**Short-term:**
- Add source citations dengan page numbers
- Implement reranking untuk better precision
- Add conversation summarization untuk long chats

**Medium-term:**
- Hybrid search (dense + sparse/BM25)
- Metadata filtering untuk more targeted retrieval
- Query expansion untuk better coverage

**Long-term:**
- Multi-modal RAG (process images/charts dari PDFs)
- Active learning dengan user feedback
- Agent-based RAG untuk complex reasoning


---
## 7. Personal Reflection

### Reflection 1: Kualitas Knowledge Base

Retrieval adalah jantung RAG. Pengalaman chunking (500 → 1000 chars) memberikan improvement lebih besar dari LLM upgrade (8B → 70B). "Garbage in, garbage out" sangat berlaku. Sekarang saya always start dengan: "How should I structure KB for optimal retrieval?" sebelum mikirin LLM.

### Reflection 2: Context-Driven Architecture

Optimal stack bukan "best tools" tapi "best fit". Groq's speed > GPT-4's capability untuk chat. MPNet multilingual > pure English untuk mixed-language. FAISS simplicity > Pinecone scalability untuk small dataset. No universal "best", only context-appropriate choices.
